In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # default is ‘last_expr’

%load_ext autoreload
%autoreload 2

In [14]:
import csv
import json
import os

import ct_utils

In [4]:
list_images = os.listdir('test_images/test_images')

In [6]:
len(list_images)

31

Command used to run the detection script in the YOLOv5 repo: 
    
```
python detect.py --weights "/Users/SiyuYang/OneDrive/CameraTraps/mdv5/yolov5_checkpoints/camonly_mosaic_xlarge_dist_5a_last.pt" --source /Users/SiyuYang/Source/repos/Microsoft/CameraTraps/test_images/test_images --imgsz 1280 --conf-thres 0.01 --device cpu --save-txt --save-conf --project mdv5 --name detect_output
```

In [9]:
detect_script_output_dir = '/Users/SiyuYang/Source/repos/my_github/yolov5/mdv5/detect_output/labels'

In [13]:
yolo_cat_map = {
    0: 1,
    1: 2,
    2: 3
}

In [20]:
images_entries = []

for image_fn in list_images:
    image_name, ext = os.path.splitext(image_fn)
    
    label_fn = image_name + '.txt'
    label_path = os.path.join(detect_script_output_dir, label_fn)
        
    detections = []
    max_conf = 0.0
    
    if os.path.exists(label_path):
        with open(label_path, newline='') as f:
            reader = csv.reader(f, delimiter=' ')
            for row in reader:
                category = yolo_cat_map[int(row[0])]

                api_box = ct_utils.convert_yolo_to_xywh([float(row[1]), float(row[2]), float(row[3]), float(row[4])])

                conf = ct_utils.truncate_float(float(row[5]), precision=4) # output precision is 6 decimal places
                max_conf = max(max_conf, conf)

                detections.append({
                    'category': str(category),
                    'conf': conf,
                    'bbox': ct_utils.truncate_float_array(api_box, precision=4)
                })
            
    images_entries.append({
        'file': image_fn,
        'max_detection_conf': max_conf,
        'detections': detections
    })
            

In [22]:
output_content = {
    'info': {
        'detector': 'megadetector_v5_camonly_mosaic_xlarge_dist_5a_last',
        'format_version': '1.1'
    },
    'detection_categories': {
        '1': 'animal',
        '2': 'person',
        '3': 'vehicle'
    },
    'images': images_entries
}

In [23]:
with open('test_images/expected_output_DRAFT.json', 'w') as f:
    json.dump(output_content, f, indent=2)